## Extracting the intensity data for analysis in MTEX

This notebook is used to extract intensity data calculated from Continuous-Peak-Fit, and to associate the intensity measurements with the correct spherical polar coordinates, so that it can be easily loaded and analysed in [MTEX](https://mtex-toolbox.github.io) - a toolbox for analysing and plotting texture in MATLAB.

In [13]:
import pathlib
import re

from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy.signal import medfilt

import yaml
from typing import Tuple
from typing import List

import continuous_peak_fit_analysis_functions as analysis
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load YAML file

The file paths and user inputs for the analysis of the Diamond 2017, Diamond 2021 and DESY 2021 experiments are included in the `yaml` configuration files, to record the inputs of the analysis.

The information in the `yaml` file can be accessed like this.

In [87]:
# to load the Diamond 2021 analysis
#config_path = "yaml/config_diamond_2021_summed.yaml"
config_path = "yaml/config_desy_2021_06.yaml"
config = analysis.get_config(config_path)

## Extracting intensity data from fit files

The peak intensity data obtained from the Fourier peak analysis in Continuous-Peak-Fit can be extracted from the .fit files using the `read_fit_results` function. This function can also be used to extract additional information from the peak fitting results, such as the peak position, half-width and pseudo-Voigt weighting.

The user inputs required to extract the intensity data are taken directly from the `yaml` configuration file, using the `extract_intensity_input` function. The user inputs include;

- Input and output file paths
- A list of lattice plane peak labels
- Azimuthal data resolution
- Image numbers

Be sure to correctly identify all peaks contained in the .fit data file for the best texture fitting in MTEX.

In [88]:
experiment_number, input_fit_path, peak_label, data_resolution, image_numbers = analysis.extract_intensity_input(config_path)

peak_position, peak_intensity, peak_halfwidth, peak_PV_weight = analysis.read_fit_results(experiment_number, 
                                                                input_fit_path, peak_label, data_resolution, image_numbers)

The experiment number is:
6

The input path to the fit file is:
../../../SXRD_analysis/desy_2021/experiment{experiment_number:02d}-deformation/fourier-peak-analysis/Ti64_Rolled_ND_Compress_910C__1-00s-1_Single-Hit_Beta-Anneal_1Cs-1_1050C_Cool_4Cs-1_Def{experiment_number:02d}_1-{image_number:05d}.fit

The peak labels for the peaks that are being used are:
['10-10', '0002', '110', '10-11', '10-12', '200', '11-20', '10-13', '20-20', '11-22', '20-21', '0004', '220', '20-22', '10-14', '310', '20-23', '21-30']

The peak labels for the peaks that are NOT being used are:
{'21-33', '321', '222', '30-30', '211', '30-31', '11-24', '21-31', '21-32', '10-15', '20-24'}

The resolution of the data is: 1 degree

The start is: 
2150
The end is 
4000
The step is 
10



  0%|          | 0/186 [00:00<?, ?it/s]

## Writing intensity data to text file for MTEX

The extracted intensity data for each of the individual lattice planes is then recorded as a pole figure, which can be loaded into MTEX. This is done using spherical polar coordinates. A notebook `NOTES-intensity-circles-to-polar-coordinates` describes in detail how to calculate the polar coordinates of different diffraction rings for different sample orientations. The `intensity_to_texture_file` function can run this conversion automatically and outputs the intensity data as a text file that can be read into MTEX. 

A final texture file is produced for each of the lattice planes and contains; 

- Polar angle (degrees)
- Azimuthal angle (degrees)
- Intensity

Currently, the function can easily convert **six** different sample orientations. 

The typical convention is that ${X = RD}$, ${Y = TD}$ and ${Z = ND}$. This gives six possible sample orientations for our sample to be aligned with respect to the horizonal and vertical direction of our detector, which gives six corresponding orientations for the planes of our circle  (of intensity points) to be aligned with respect to the horizonal and vertical direction of our detector;
    
| Sample Name | Horizontal-Vertical Alignment (Rolled Sample) | Horizontal-Vertical Alignment (Cartesian) |
| -- | --------- | --- |
| S3 | RD-TD     | X-Y |
| S2 | RD-ND     | X-Z |
| S1 | TD-ND     | Y-Z |
| S5 | RD-TD45ND | Tilt between X-Y and X-Z |
| S4 | TD-RD45ND | Tilt between Y-X and Y-Z |
| S6 | TD45RD-ND | Tilt between X-Z and Y-Z |

An additional **three** sample orientations can also be used, which rotate the horizontal and vertical axes;

| Sample Name | Horizontal-Vertical Alignment (Rolled Sample) | Horizontal-Vertical Alignment (Cartesian) |
| --- | --------- | --- |
| S3R | TD-RD     | Y-X |
| S2R | ND-RD     | Z-X |
| S1R | ND-TD     | Z-Y |


[TODO] - Extend the script to cover all orientation space *(not just 45 degrees)*.

[TODO] - Extend script to include correction for 2${\theta}$ scattering angle. *Due to smoothing of the data in MTEX, this is not thought to be a problem for 2${\theta}$ angles less than 10 degrees, as the correction factor is dependant on ${\theta}$, which is less than 5 degrees, and a 10 degree half-width is used to produce the most accurate texture fit.*

In [89]:
intensity_type = 'raw'

plane, output_text_path = analysis.extract_writing_intensity_input(config_path)

analysis.intensity_to_texture_file(experiment_number, intensity_type, output_text_path, peak_intensity, 
                                   peak_label, plane, data_resolution, image_numbers)

The sample orientation type is:
S1R

The sample plane type is:
ZY

The output path to the text file for MTEX is:
../../../SXRD_analysis/desy_2021/experiment{experiment_number:02d}-deformation/fourier-peak-analysis-texture/{experiment_number:02d}-stage-1-{intensity_type}-intensities/{experiment_number:02d}_{image_number:05d}_peak_intensity_{label}.txt



  0%|          | 0/186 [00:00<?, ?it/s]

Written '186' set of raw intensity .txt data files to: '../../../SXRD_analysis/desy_2021/experiment06-deformation/fourier-peak-analysis-texture/06-stage-1-raw-intensities/06_04000_peak_intensity_21-30.txt'.


### Loading the text files into MTEX

These texture files can be loaded into MTEX using a generic file upload. A generic script can be chosen in the `Import Wizard` by selecting all of the data files, for all of the lattice planes, then choosing `Spherical (Polar, Azimuth)` and `Degree` as the `Angle Convention`. An example MTEX script has already been created to plot the texture for the Diamond 2017, Diamond 2021 and DESY 2021 intensity text files, which can be found in the `mtex` folder. This uses the following `generic` command to load the data as a pole figure;

- pf = PoleFigure.load(fname,h,CS,SS,'interface','generic',...
    'ColumnNames', { 'Polar Angle' 'Azimuth Angle' 'Intensity'});
    
Where `fname` is a list of all of the lattice plane text files,  `h` is the Miller indices of the lattice planes, `CS` is the crystal symmetry and `SS` is the specimen symmetry.

A loop can be used to iterate through large datasets to calculate how the texture changes over time.

## Combine intensities from different samples / orientations

A single sample orientation can give a good approximation of the texture calculated using MTEX. But, there is still a lot of missing information, since the diffraction pattern rings are only a very small subset of grains that happen to fulfill the diffraction scattering condition.

The texture accuracy can be signifcantly improved by combining the intensity results from different sample orientations. In our study of room temperature texture in Ti-64 materials at room temperature, we found **2** sample orientations gives a much better approximation of the ${\alpha}$ and ${\beta}$ texture, as well as significantly improving the accuracy texture strength. **3** samples improved the accuracy further. And **6** samples gives an almost exact match in the texture strength to electron backscatter diffraction (EBSD) results.

To create a text file with multiple sample orientations, we simply have to sum up all of the intensity and spherical polar coordinates from the individual text files. The `extract_combine_intensity_input` is used to select the input and output paths from a separate `yaml` configuration file to run this analysis, and the `combine_texture_files` function writes out the text file. This creates one larger text file with greater pole figure coverage, that can easily be loaded into MTEX.

In [ ]:
# to load a combined Diamond 2021 analysis
config_combine_path = "yaml/config_diamond_2021_combined.yaml"

In [ ]:
input_text_path_combine, output_text_path_combine, intensity_type, experiment_numbers_combine, image_numbers_combine = analysis.extract_combine_intensity_input(config_combine_path)

analysis.combine_texture_files (input_text_path_combine, output_text_path_combine, intensity_type, 
                                powder_peak_label, experiment_numbers_combine, image_numbers_combine)

## Calibrating intensity data to powder measurements

In some cases it may be necessary to calibrate the intensity measurements to a calibration sample, with a random texture.

In our experiments of ${\alpha}$ and ${\beta}$ texture in Ti-64 we did not find this was necessary to produce accurate calculation of texture and texture strength in MTEX. The texture results calculated directly from intensity measurements exactly match texture calculated from calibrated intensities, which is due to the way MTEX uses pole figure intensity measurements to calculate texture.

In any case, if calibrating intensity measurements to a randomly textured powder sample is required, the following workflow allows us to do this.

First, give the path to the powder configuration file.

In [ ]:
# to load the Diamond 2021 powder analysis
config_powder_path = "yaml/config_diamond_2021_powder.yaml"

Then, extract the powder intensity measurements.

*Note, some lattice plane peaks may not be possible to index in the powder sample.*

In [ ]:
powder_experiment_number, powder_input_fit_path, powder_peak_label, powder_data_resolution, powder_image_numbers = analysis.extract_powder_intensity_input(config_powder_path)

powder_peak_position, powder_peak_intensity, powder_peak_halfwidth, powder_peak_PV_weight = analysis.read_fit_results(
                                                powder_experiment_number, powder_input_fit_path, 
                                                powder_peak_label, powder_data_resolution, powder_image_numbers)

Then, correct the intensity values in the data, by dividing by an average intensity value measured in the randomly textured powder sample, using the `calibrate_intensity_to_powder` function.

*Note, the average intensity value will be different for each of the different lattice plane peaks*

This produces intensity values as multiples of a random intensity, which can then be saved to a text file for analysing in MTEX. The data is saved to a separate folder labelled `powder-corrected`.

In [ ]:
corrected_peak_intensity = analysis.calibrate_intensity_to_powder(peak_intensity, powder_peak_intensity,
                                                         powder_peak_label, image_numbers, powder_start=1)

intensity_type = 'powder-corrected'
analysis.intensity_to_texture_file(experiment_number, intensity_type, output_text_path, corrected_peak_intensity, 
                                   powder_peak_label, plane, data_resolution, image_numbers)

The corrected intensity data can also be combined from separate sample orientations, using the same function as before. This data is also saved to a separate folder labelled `powder-corrected`.

In [ ]:
intensity_type = 'powder-corrected'
analysis.combine_texture_files (input_text_path_combine, output_text_path_combine, intensity_type, 
                                powder_peak_label, experiment_numbers_combine)